In [1]:
import trimesh

scene = trimesh.Scene()

In [20]:
mesh_demo, mesh_aligned

In [ ]:
import numpy as np

# mesh_demo = trimesh.load('/home/s94zalek_hpc/shape_matching/data/FAUST_r/off/tr_reg_091.off', process=True, validate=True)
mesh_demo = trimesh.load('/home/s94zalek_hpc/baselines/NICP/output/1ljjfnbx/demo/FAUST_r_tr_reg_080/aligned.ply', process=True, validate=True)
mesh_aligned = trimesh.load('/home/s94zalek_hpc/baselines/NICP/output/1ljjfnbx/demo/FAUST_r_tr_reg_080/out_ss_cham_0.ply', process=True, validate=True)

# mesh_demo = trimesh.load('/home/s94zalek_hpc/baselines/NICP/output/1ljjfnbx/demo/test_scan_044/out_ss.ply')

mesh_aligned.vertices += np.array([0, 0, 0.2])

scene.geometry.clear()

scene.add_geometry(mesh_demo)
scene.add_geometry(mesh_aligned)

scene.show()

In [24]:
dataset_name = 'SHREC19_r'

base_dir = f'/home/s94zalek_hpc/shape_matching/data/{dataset_name}/off'
target_dir = f'/home/s94zalek_hpc/baselines/NICP/demo'

# for each file in base_dir, make a folder in target_dir with name dataset_name_file_name (without extension) and copy the file there. 

import os

for file_name in sorted(os.listdir(base_dir)):
    if file_name.endswith('.off'):
        file_name_no_ext = file_name[:-4]
        os.makedirs(f'{target_dir}/{dataset_name}_{file_name_no_ext}', exist_ok=True)
        os.system(f'cp {base_dir}/{file_name} {target_dir}/{dataset_name}_{file_name_no_ext}/{dataset_name}_{file_name}')
            
        


### Evaluate Geo Error

In [3]:
import numpy as np
import my_code.diffusion_training_sign_corr.data_loading as data_loading
import yaml
from tqdm import tqdm
import metrics.geodist_metric as geodist_metric
from utils.shape_util import compute_geodesic_distmat
import torch


dataset_name = 'FAUST_r_pair'

single_dataset, pair_dataset = data_loading.get_val_dataset(
    dataset_name, 'test', 128, preload=False, return_evecs=True, centering='bbox'
)


In [17]:
dist_mat_list = []

for i in tqdm(range(len(single_dataset))):
    data_i = single_dataset[i]
    
    dist_i = torch.tensor(
        compute_geodesic_distmat(data_i['verts'].numpy(), data_i['faces'].numpy())    
    )
    dist_mat_list.append(dist_i)

In [10]:
import utils.fmap_util as fmap_util

def get_reg(dataset_name, shape_name):

    mesh_aligned = trimesh.load(
        f"/home/s94zalek_hpc/baselines/NICP/output/1ljjfnbx/demo/{dataset_name[:-5]}_{shape_name}/aligned.ply", process=False, validate=False
        )
    mesh_fitted = trimesh.load(
        f"/home/s94zalek_hpc/baselines/NICP/output/1ljjfnbx/demo/{dataset_name[:-5]}_{shape_name}/out_ss_cham_0_+D.ply", process=False, validate=False
    )

    corr = fmap_util.nn_query(
        torch.tensor(mesh_aligned.vertices, dtype=torch.float32),
        torch.tensor(mesh_fitted.vertices, dtype=torch.float32),
        dim=-2)
    
    # print(mesh_aligned.vertices.shape, mesh_fitted.vertices.shape, corr.shape)
    
    return corr, mesh_aligned, mesh_fitted

In [26]:
geo_err_list = []

iterator = tqdm(range(len(pair_dataset)), disable=True)

for i in iterator:
    data_i = pair_dataset[i]

    reg_first, mesh_aligned_first, mesh_fitted_first = get_reg(dataset_name, data_i['first']['name'])
    reg_second, mesh_aligned_second, mesh_fitted_second = get_reg(dataset_name, data_i['second']['name'])

    Cxy_lstsq = torch.linalg.lstsq(
        data_i['second']['evecs'][reg_second],
        data_i['first']['evecs'][reg_first]
        ).solution

    p2p = fmap_util.fmap2pointmap(
        Cxy_lstsq,
        data_i['first']['evecs'],
        data_i['second']['evecs'],
        )

    dist_first = dist_mat_list[data_i['first']['id']]

    geo_err = geodist_metric.calculate_geodesic_error(
        dist_first, data_i['first']['corr'], data_i['second']['corr'], p2p, return_mean=True
    )*100
    
    geo_err_list.append(geo_err)
    
    iterator.set_description(f'geo_err: {geo_err:.2f}')
    
    print(i, geo_err)

In [16]:
dist_first = torch.tensor(
        compute_geodesic_distmat(data['first']['verts'].numpy(), data['first']['faces'].numpy())    
    )

geo_err = geodist_metric.calculate_geodesic_error(
    dist_first, data['first']['corr'], data['second']['corr'], p2p, return_mean=True
)
print(geo_err)

In [27]:
import my_code.utils.plotting_utils as plotting_utils

scene.geometry.clear()

plotting_utils.plot_p2p_map(
    scene,
    
    data['first']['verts'], data['first']['faces'],
    data['second']['verts'], data['second']['faces'],
    
    
    p2p,
    axes_color_gradient=[0, 1],
    base_cmap='hsv'
)

scene.show()

In [7]:
import importlib
importlib.reload(plotting_utils)

In [24]:
data_i = pair_dataset[9]

reg_first, mesh_aligned_first, mesh_fitted_first = get_reg(dataset_name, data_i['first']['name'])
reg_second, mesh_aligned_second, mesh_fitted_second = get_reg(dataset_name, data_i['second']['name'])


In [25]:
import my_code.utils.plotting_utils as plotting_utils

scene.geometry.clear()

plotting_utils.plot_p2p_map(
    scene,
    
    # data['first']['verts'], data['first']['faces'],
    # data['second']['verts'], data['second']['faces'],
    # verts_ref, faces_ref,
    
    # mesh_aligned_first.vertices, mesh_aligned_first.faces,
    # mesh_fitted_first.vertices, mesh_fitted_first.faces,
    # reg_first,
    
    # second
    mesh_aligned_second.vertices, mesh_aligned_second.faces,
    mesh_fitted_second.vertices, mesh_fitted_second.faces,
    reg_second,
    
    axes_color_gradient=[0, 1],
    base_cmap='hsv'
)

scene.show()

In [67]:
verts_ref.shape, reg_first.shape

In [12]:
import my_code.datasets.preprocessing as preprocessing

# reference mesh
mesh_ref = trimesh.load('/home/s94zalek_hpc/baselines/NICP/output/1ljjfnbx/demo/FAUST_r_tr_reg_080/out_ss_cham_0.ply', process=False, validate=False)

verts_ref = torch.tensor(mesh_ref.vertices, dtype=torch.float32)
faces_ref = torch.tensor(mesh_ref.faces, dtype=torch.int64)

verts_ref = preprocessing.normalize_face_area(verts_ref, faces_ref)

dist_ref = torch.tensor(
        compute_geodesic_distmat(verts_ref.numpy(), faces_ref.numpy())    
    )

In [24]:
print(reg_first.shape, reg_first.min(), reg_first.max())
print(data['first']['corr'].shape, data['first']['corr'].min(), data['first']['corr'].max())

In [30]:
dist_ref[reg_first_ref, reg_second_ref]

In [13]:
reg_first_ref = reg_first[data['first']['corr']]
reg_second_ref = reg_second[data['second']['corr']]

geo_err = dist_ref[reg_first_ref][reg_second_ref].mean() * 100
geo_err

In [35]:
reg_first_ref.shape

In [ ]:
import my_code.utils.plotting_utils as plotting_utils

scene.geometry.clear()

plotting_utils.plot_p2p_map(
    scene,
    
    # data['second']['verts'], data['second']['faces'],
    verts_ref, faces_ref,
    data['first']['verts'], data['first']['faces'],
    
    reg_first_ref,
    axes_color_gradient=[0, 1],
    base_cmap='hsv'
)

scene.show()

In [51]:
reg_first

In [52]:
data['first']['corr'].min(), data['first']['corr'].max()

In [53]:
data['first']['verts'].shape